In [ ]:
from google.colab import drive
import urllib
import os
import json
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/STAT5106_asg4')
import GeoAPI_functions_for_assignment as geoapi

Mounted at /content/drive


In [ ]:
#Q1
def getRoute(origin, destination):
  try:
    resp = geoapi.getGeojs_route(origin, destination)
    routes = resp["routes"]
    distance = routes[0]["distance"] / 1000
    duration = routes[0]["duration"] / 60
    return {'Dist': distance, 'Time': duration}
  except:
    return None

#UAT #1
getRoute('Ocean Park', 'The Chinese University of Hong Kong')

{'Dist': 27.1549, 'Time': 35.255}

In [ ]:
#Q2
tunnel_fares = geoapi.getTunnelFares()
def getRoute(origin, destination):
  try:
    resp = geoapi.getGeojs_route(origin, destination)
    routes = resp["routes"]
    tunnels = list(map(lambda x: x[0],
                       filter(lambda x: x, #filter invalid values
                              map(lambda x: [s for s in tunnel_fares.keys() if s in x["name"].replace("-", " ")], #map the tunnel from the response object
                                  routes[0]["legs"][0]["steps"])
                              )
                       )
    )
    distance = routes[0]["distance"] / 1000
    duration = routes[0]["duration"] / 60
    return {'Dist': distance, 'Time': duration, 'TollRoads': sorted(list(set(tunnels)))}
  except:
    return None

def getRoute_offline(txtfile_name):
  try:
    with open(txtfile_name,"r") as f:
      s = f.read()
    js = json.loads(s)
  except:
    js = None
  return js


#UAT #2
print(getRoute('Ocean Park', 'The Chinese University of Hong Kong'))
#UAT #3
print(getRoute('Cyberport', 'The Chinese University of Hong Kong'))
#UAT #4
print(getRoute('Cityplaza', 'The Chinese University of Hong Kong'))
#UAT #5
print(getRoute('Tai Po Market Station', 'The Chinese University of Hong Kong'))

{'Dist': 27.1549, 'Time': 35.255, 'TollRoads': ['Aberdeen Tunnel', 'Cross Harbour Tunnel', 'Lion Rock Tunnel']}
{'Dist': 26.5033, 'Time': 32.675, 'TollRoads': ['Tsing Sha Highway', 'Western Harbour Crossing']}
{'Dist': 24.4894, 'Time': 30.15, 'TollRoads': ['Eastern Harbour Crossing', "Tate's Cairn Tunnel"]}
{'Dist': 7.5811, 'Time': 12.711666666666668, 'TollRoads': []}


In [ ]:
#Q3
def getRawTaxiFare(dist_km, TaxiType='urban'):
  threshold_longdist = {'urban': 9.0, 'nt': 8.0, 'lantau': 20.0}
  threshold_fare = {'urban': 83.5, 'nt': 65.5, 'lantau': 154.0}
  
  start_fare = {'urban': 24.0, 'nt': 20.5, 'lantau': 19.0}
  tick_fare = {'urban': 1.7, 'nt': 1.5, 'lantau': 1.5}
  longdist_tick_fare = {'urban': 1.2, 'nt': 1.2, 'lantau': 1.4}
  
  TaxiType = TaxiType.lower()
  fare = None
  try:
    if dist_km > threshold_longdist[TaxiType]:
      # if fare > 83.5 or km > 9.0, for each tick = $1.2
      n_tick = (dist_km - threshold_longdist[TaxiType]) *1000 // 200
      fare = threshold_fare[TaxiType] + n_tick * longdist_tick_fare[TaxiType]
    elif dist_km > 2.0:
      n_tick = (dist_km - 2.0)*1000 // 200
      fare = start_fare[TaxiType] + n_tick * tick_fare[TaxiType]
    else:
      # meter not tick
      n_tick = 0
      fare = start_fare[TaxiType]
  except:
      print('Something goes wrong !')
      fare = None

  return fare

def TaxiFare(origin, destination, TaxiType='urban', useFile = None):
  tunnel_fares = geoapi.getTunnelFares()
  fare = None
  try:
    r = getRoute(origin, destination, useFile)
    rawFare = getRawTaxiFare(r["Dist"], TaxiType)
    selectedTunnelFare = sum({ k: tunnel_fares[k] for k in r["TollRoads"] }.values())
    fare = rawFare + selectedTunnelFare
  except:
    fare = None
  return fare

#UAT #6
print(TaxiFare('Ocean Park', 'The Chinese University of Hong Kong'))

#UAT #7
print(TaxiFare('Tai Po Market Station', 'The Chinese University of Hong Kong', TaxiType='NT'))

#UAT #8
print(TaxiFare('Tsuen Wan Station', 'The Chinese University of Hong Kong', TaxiType='NT'))

224.5
61.0
118.5
